# Training notebook

In [1]:
import utils
import model

In [2]:
# set dataset to use
dataset_name = 'facades'

In [3]:
# instantiate datasets and dataloaders
datasets = {mode: utils.ImageDataset(dataset_name = dataset_name, transform=utils.data_transforms[mode], mode = mode) for mode in ['train', 'val']}
dataloaders = {mode: utils.DataLoader(datasets[mode], batch_size=1, shuffle=True) for mode in ['train', 'val']}

In [4]:
# instantiate model
p2p = model.Pix2Pix().to(utils.device)

In [5]:
## In the paper the default number of epoch is 200
utils.train(model = p2p, n_epochs = 1, dataloader = dataloaders['train'])

Epoch 1/1
----------
400/400: [===============================>] - ETA 0.8sss
Loss: 88.4661, generator loss: 16364.0982, discriminator loss: 88.4661.
Training complete in 0m 19s


In [6]:
input, real = next(iter(dataloaders['val']))

In [7]:
utils.generate_images(model = p2p, input = input, real = real)

TypeError: forward() got an unexpected keyword argument 'training'